<a href="https://colab.research.google.com/github/rabierp/GCP-ML-APIs-Labs/blob/main/Vision_NLP_Translate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialisation de votre environnment Projet

## Vérifier votre environnement

Vérifiez le compte (l'identité) utilisée par la VM qui exécute ce Notebook IPython (la commande ci-dessous doit renvoyer un compte de service de type _Compute Engine_) : 

In [ ]:
!gcloud config get-value core/account

Vérifiez que vous êtes dans le bon projet GCP (la commande ci-dessous doit renvoyer le nom de projet temporaire créé par Qwiklabs) :

In [ ]:
!gcloud config get-value core/project

Assignez l'identifiant de votre projet GCP à la variable d'environnment $GOOGLE_CLOUD_PROJECT :

In [ ]:
PROJECT = !gcloud config get-value core/project
%env GOOGLE_CLOUD_PROJECT {PROJECT[0]}

## Vérifier que les APIs Vision et Natural-Language sont bien activées

In [ ]:
!gcloud services list --enabled --filter=vision

In [ ]:
!gcloud services list --enabled --filter=language

In [ ]:
!gcloud services list --enabled --filter=translate

## Activer les APIs éventuellement non activées
Si certaines APIs ne sont pas encore activées, retournez sur l'interface web de la console GCP, cliquez sur __Navigation Menu__ (icône dite "Burger menu") > __APIs & Services__ > __Library__.

Cherchez la ou les API(s) non encore activées et cliquez sur __Enable__.

## Créer votre compte de service pour appeler les APIs

Utilisez la commande _gcloud_ pour créer le compte de service que vous allez utiliser dans la suite de ce lab pour appeler les APIs :

In [ ]:
!gcloud iam service-accounts create my-lab-sa --display-name "my lab service account"

Utilisez la commande _gcloud_ pour télécharger en local la clé d'authentification du compte de service :

In [ ]:
!gcloud iam service-accounts keys create key.json --iam-account my-lab-sa@${GOOGLE_CLOUD_PROJECT}.iam.gserviceaccount.com

Créez une variable qui contient le nom complet (chemin inclus) du fichier téléchargé :

In [ ]:
path = !pwd
keyfilename = path[0] + "/key.json"

Et enregistrez le nom complet de ce fichier dans la variable d'environnement $GOOGLE_APPLICATION_CREDENTIALS :

In [ ]:
%env GOOGLE_APPLICATION_CREDENTIALS {keyfilename}

**Vous êtes maintenant prêt à appeler les APIs de Machine Learning de GCP !!!**

# Détection de "Point de repère" par appel de l'API Vision avec _curl_

La détection de point de repère ("_LANDMARK DETECTION_") dans une image est une des fonctionnalités de l'API Cloud Vision.

Dans cette section, vous allez analyser [cette image](https://storage.googleapis.com/cloud-vision-codelab/eiffel_tower.jpg), préalablement chargée pour vous dans un _bucket_ **Google Cloud Storage** (vous pouvez cliquer sur le lien pour visualiser l'image).

## Préparation du corps de la requête API

Vous allez d'abord construire la requete HTTP pour appeler l'API Vision, puis envoyer cette requête avec la commande en ligne curl (commande shell Linux / Unix).

Ecrivez le corps de votre requete en JSON dans le fichier request.json :

In [ ]:
%%bash
cat <<EOF > request.json
{
  "requests": [
    {
      "image": {
        "source": {
              "gcsImageUri": "gs://cloud-vision-codelab/eiffel_tower.jpg"
          }
      },
      "features": [
        {
          "maxResults": 10,
          "type": "LANDMARK_DETECTION"
        },
      ]
    }
  ]
}
EOF

Vérifiez le contenu du fichier 'request.json' :

In [ ]:
!cat request.json

## Exécution de l'appel API avec la commande _curl_

La commande _curl_ est une commande en ligne (CLI) disponible depuis le shell d'un système Linux / Uni, voire Windows si les outils adéquats ont été préalablement installés.

In [ ]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @request.json \
https://vision.googleapis.com/v1/images:annotate

Le résultat affiché ci-dessus doit être le corps de la réponse à la requête HTTPS, sous forme d'objet _json_.

Vous êtes invités à le regarder attentivement pour en comprendre la structure.

# Détection de libellés sur une image avec l'API Cloud Vision

A partir de cette section, vous allez utiliser la bibliothèque cliente Google Vision pour Python, méthode la plus naturelle dans ce langage (ou _"pythonic"_ pour les geeks).

Dans cette section, vous allez utiliser la fonctionnalité de détection de libellés ("_LABEL_DETECTION_") de l'API Cloud Vision, sur l'[image suivante](https://storage.googleapis.com/ml-api-codelab/birds.jpg), préalablement chargée pour vous dans un bucket **Google Cloud Storage** (vous pouvez cliquer sur le lien pour visualiser l'image).

Vérifiez que la bibliothèque client Vision pour Python est bien installée dans votre environnement :

In [ ]:
!pip list | grep "google-cloud-vision"

Si la commande précédente renvoie une réponse vide, installez la bibliothèque client Cloud Vision pour Python :

In [ ]:
!pip install google-cloud-vision

Ensuite, vous pouvez importer le package dans votre code :

In [ ]:
from google.cloud import vision

Puis initialiser le client :

In [ ]:
vision_client = vision.ImageAnnotatorClient()

Déclarez l'image que vous souhaitez analyser (ou annoter) :

In [ ]:
image_uri = 'gs://ml-api-codelab/birds.jpg'
image = vision.types.Image()
image.source.image_uri = image_uri

Et enfin appeler l'API pour obtenir l'analyse de l'image :

In [ ]:
response = vision_client.label_detection(image=image)

Vous pouvez maintenant parcourir les libellés (_LABELS_) détectés par l'API, qui sont maintenants stockés dans le dictionnaire '_response_' :

In [ ]:
print('Libellés détectés (et scores de confiance associés):')
print('=' * 52)
for label in response.label_annotations:
    print(label.description, '(%.2f%%)' % (label.score*100.))

La documentation de l'API Vision est [ici](https://cloud.google.com/vision/docs).

Les guides pratiques [là](https://cloud.google.com/vision/docs/how-to) (utile pour avoir une compréhension globale de l'ensemble des fonctionnalités de l'API).

La documentation de référence pour la version Python est à [cette adresse](https://googleapis.dev/python/vision/latest/index.html).

# Détection de texte sur une image avec l'API Cloud Vision
L'objectif de est de vous apprendre à utiliser la fonctionnalité de **reconnaissance optique des caractères (OCR)** de l'API Cloud Vision, ane l'appliquant à [cette image](https://storage.googleapis.com/pr-ext-shares/pancarte_Bien_Public.jpeg), préalablement chargée pour vous dans un bucket Google Cloud Storage (vous pouvez cliquer sur le lien pour visualiser l'image).

Le module google.cloud.vision a déjà été importé dans la section précédente.

Le client a déjà été initialisé.

Vous pouvez donc dès à présent déclarer l'image à analyser, et appeler l'API :

In [ ]:
image_uri = 'gs://pr-ext-shares/pancarte_Bien_Public.jpeg'
image = vision.types.Image()
image.source.image_uri = image_uri

response = vision_client.text_detection(image=image)

La boucle suivante va parcourir les éléments de texte détectés dans l'image, et afficher leur position sur cette dernière :

In [ ]:
for text in response.text_annotations:
    print('=' * 30)
    print(text.description)
    vertices = ['(%s,%s)' % (v.x, v.y) for v in text.bounding_poly.vertices]
    print('bounds:', ",".join(vertices))

*************************

Afin de préparer la prochaine section, récupérez dans la variable '_chaine_' tout le texte identifié sur l'image, qui se trouve dans le 1er élément de la liste '_text_annotations_' : 

In [ ]:
chaine = response.text_annotations[0].description
print(chaine)

# Utilisation de l'_API Natural Language_

## Vérifiez votre environnement Python

Vérifiez que la bibliothèque client _Natural Language_ pour Python est bien installée dans votre environnement :

In [ ]:
!pip list | grep "google-cloud-language"

Si la commande précédente renvoie une réponse vide, installez le package python de la bibliothèque client :

In [ ]:
!pip install google-cloud-language -U

## Initialisez votre code

Importez le module Python et initialisez le client :

In [ ]:
from google.cloud import language_v1
nlu_client = language.LanguageServiceClient()

## Analyser les entités présentes dans le texte

Déclarez le texte à analyser :

In [ ]:
type_ = language_v1.types.Document.Type.PLAIN_TEXT
langue = "fr"
document = {"content": chaine, "type": type_, "language": langue}

Appelez l'API :

In [ ]:
response = nlu_client.analyze_entities(document=document)

Affichage de la réponse :

In [ ]:
for entity in response.entities:
    print('=' * 20)
    print('         name: {0}'.format(entity.name))
    print('         type: {0}'.format(entity.type))
    print('     metadata: {0}'.format(entity.metadata))
    print('     salience: {0}'.format(entity.salience))

## Analyse de la syntaxe du texte

Appelez l'API :

In [ ]:
response = nlu_client.analyze_syntax(document=document)

Affichage de la réponse :

In [ ]:
fmts = "{:10}: {}"
print(fmts.format("sentences", len(response.sentences)))
print(fmts.format("tokens", len(response.tokens)))
for token in response.tokens:
#    print(fmts.format(token.part_of_speech.tag, token.text.content))
    tag_name = language_v1.enums.PartOfSpeech.Tag(token.part_of_speech.tag)
    content = token.text.content
    print(u'{!r}: {}'.format(tag_name, content))

# Identification de texte sur une autre image et appel de l'_API Translation_

## Appelez l'API Vision pour détecter le texte sur l'image

Vous allez à nouveau utiliser la fonctionnalité de **reconnaissance optique des caractères (OCR)** de l'API Cloud Vision, en l'appliquant cette fois à [cette image](https://storage.googleapis.com/cloud-vision-codelab/otter_crossing.jpg), préalablement chargée pour vous dans un bucket Google Cloud Storage (vous pouvez cliquer sur le lien pour visualiser l'image).

Le module google.cloud.vision a déjà été importé dans la section précédente.

Le client a déjà été initialisé.

Vous pouvez donc dès à présent compléter le code suivant pour déclarer l'image à analyser, et appeler l'API :

In [ ]:
image_uri = 'XXXXXXXXXXXXXXXXXXXXX'
image = vision.types.Image()
image.source.image_uri = image_uri

response = vision_client.<UTILISER_LA_BONNE_FONCTIONNALITE>(image=image)

Puis, complétez le code suivant pour afficher les mots reconnus :

In [ ]:
for text in response.<LA_BONNE_PROPRIETE>:
    print('=' * 30)
    print(text.<LA_BONNE_PROPRIETE>)
    vertices = ['(%s,%s)' % (v.x, v.y) for v in text.bounding_poly.vertices]
    print('bounds:', ",".join(vertices))

## Traduire le texte avec l'API Translation

Stocker le texte précédent dans une variable '_chaine_' :

In [ ]:
chaine = response.text_annotations[0].description
print(chaine)

Initialisez le client Python de l'API Translation :

In [ ]:
from google.cloud import translate_v2 as translate

translate_client = translate.Client()

Préparez la requête / spécifiez la langue cible :

In [ ]:
"""Translates text into the target language.

Target must be an ISO 639-1 language code.
See https://cloud.google.com/translate/docs/reference/rest/v3/SupportedLanguages
"""
target = "fr"

Exécutez l'appel API :

In [ ]:
# Text can also be a sequence of strings, in which case this method
# will return a sequence of results for each text.
result = translate_client.translate(chaine, target_language=target)

Affichez le résultat :

In [ ]:
print(u"Text: {}".format(result["input"]))
print(u"Translation: {}".format(result["translatedText"]))
print(u"Detected source language: {}".format(result["detectedSourceLanguage"]))